<h1 align="center">Trabalho 2 - Aprendizagem de Máquina</h1>

## Andre Brun
### Daniel Boll & Mateus Karvat
---


Inicialmente, importamos as bibliotecas necessárias.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib import style, cm
import matplotlib.tri as mtri
from mpl_toolkits.mplot3d import Axes3D
import time

# Analitics
from sklearn.metrics import accuracy_score, silhouette_score, pairwise_distances
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance

# Clusters
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering

# Configurations
style.use('ggplot')
%matplotlib qt
np.set_printoptions(precision=3, suppress=True)
pd.set_option("display.precision", 3)

Em seguida, carregamos a base de dados.

In [2]:
data = pd.read_csv('./Base9.csv')

# Manteremos uma cópia dos dados originais
# para garantia
raw_data = data.copy()

Separando as coordenadas x e y na variável ***X*** e as labels na variável ***label***.

In [3]:
X = np.array(data.values[:, :2])
label = np.array(data.values[:, 2])

Após carregar a base e separá-la, plotamos o *dataset* com as cores verde para a primeira classe e vermelha para a segunda. O dataset é popularmente chamado de "Banana", de modo que ocasionalmente nos referimos a cada cluster como uma "banana".

In [4]:
colors = ["g.", "r.", "c.", "b."] ## No caso de ter até 4 classes
for i in range(len(X)):
    plt.plot(X[i][0], X[i][1], colors[int(label[i])])
plt.show()

<div align="center">
    <img src="./images/dataset_plot.png" width=300/>
</div>

---
## Metricas de avaliação 

Funções implementadas por nós para as métricas de avaliação não implementadas pelas bibliotecas utilizadas. A métrica de Silhueta é utilizada a partir de função pré-pronta do SciKit Learn.

### Coesão

In [4]:
def cohesion_score(X, labels):
    """
    N - N dentro de uma mesma label
    pra cada label definida pelo classificador
    """
    each_label = np.unique(labels)
    total = 0

    # para cada label, obtemos a matriz subX, que contém as coordenadas dos pontos daquelas labels
    for lab in each_label:
        if lab!=-1:
            indices = np.where(labels == lab)
            indices = indices[0]
            subX = np.take(X, indices, axis=0)

            # verificamos a distância euclidiana entre todos os pares de pontos em subX
            total += np.sum(pairwise_distances(subX, metric='sqeuclidean', n_jobs=-1))

    # a fim de reduzir a dimensão do resultado, retornarmos sua raiz quadrada
    return np.sqrt(total)

### Separação

In [5]:
def separation_score(X, labels):
    """
    N - M
    """
    each_label = np.unique(labels)
    total = 0
    
    # assim como na coesão, obtemos a matriz subX com os pontos pertencentes a uma mesma label. Porém, aqui precisamos da matriz subY com os pontos pertencentes às demais labels.
    for lab in each_label:
        if lab!=-1:
            indices_x = np.where(labels == lab)
            indices_x = indices_x[0]

            indices_y = np.where(np.logical_and(labels != lab, labels!=-1))
            indices_y = indices_y[0]
            
            subX = np.take(X, indices_x, axis=0)
            subY = np.take(X, indices_y, axis=0)

            # calculamos a distância entre os pontos de subX e os pontos de subY
            total += np.sum(pairwise_distances(subX, subY, metric='sqeuclidean', n_jobs=-1))
    return np.sqrt(total)

### Entropia

In [6]:
def entropy_score(X, label_class, label_dataset):
    """
        banana 🍌
        B1 = label_dataset == 1
        B2 = label_dataset == 0
    """
    cluster_labels = np.unique(label_dataset)
    total_entropy = 0
    for label in cluster_labels:
        cluster_entropy = 0
        
        # Cluster indices tem o índice
        # de uma banana (dataset original)
        cluster_indices = np.where(label_dataset == label)
        cluster_indices = cluster_indices[0]

        # Tem todas as instâncias da label_class
        # dentro da banana atual (cluster_indices)
        cluster_classes = np.take(label_class, cluster_indices)

        # probs inicialmente armazena a quantidade de pontos com cada classificação e posteriormente é dividido pelo número de pontos naquele cluster, de modo a armazenar as probabilidades em cada cluster
        classes, probs = np.unique(cluster_classes, return_counts=True)
        cluster_sum = np.sum(probs)
        if classes[0]==-1:
            probs = probs[1:]
        probs = probs / cluster_sum

        # para cada p possível, calcula-se o elemento do somatório da entropia
        for p in probs:
            cluster_entropy += p * np.log2(p)

        # a entropia daquele cluster é adicionada à entropia total (com sinal negativo)
        total_entropy -= cluster_entropy

    # é feita a média das entropias
    total_entropy /= len(cluster_labels)
    return total_entropy

---
## Função para plotar gráficos tridimensionais

In [7]:
def plotScore3D(xp, yp, zp, title, x_axis_name, y_axis_name):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    x, y = np.meshgrid(xp, yp)
    z = zp

    surf = ax.plot_surface(y, x, z, cmap=cm.coolwarm,
                        linewidth=0, antialiased=True)           
                        
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
    ax.set_xlabel(x_axis_name)
    ax.set_ylabel(y_axis_name)
    
    # Adiciona um colorbar que mapeia os valores para diferentes cores
    fig.colorbar(surf, shrink=0.5, aspect=5)
    plt.title(title)
    plt.show()

---

## KMeans

Para o classificador KMeans, os parâmetros testados são:
* Número de centróides (variável "n_clusters") variando de 2 a 8
* Número de iterações para convergências (variável "max_iter") variando de 1 a 10

In [8]:
# o dicionário kmc_parameters armazena os possíveis valores de parâmetros para o classificador
kmc_parameters = {
    "n_clusters": [i for i in range(2, 9)],
    "max_iter": [j for j in range(1, 11)]
}

# as variáveis com sufixo "_size" aramazenam o tamanho dos vetores no dicionário
cluster_size = np.shape(kmc_parameters['n_clusters'])[0]
iter_size    = np.shape(kmc_parameters['max_iter'])[0]

# com as variáveis de sufixo "_size", criamos as matrizes que armazenarão a pontuação do classificador para todas as possíveis combinações de parâmetros conforme as diferentes métricas de avaliação
kmc_score_matrix_cohesion   = np.zeros((cluster_size, iter_size))
kmc_score_matrix_separation = np.zeros((cluster_size, iter_size))
kmc_score_matrix_entropy    = np.zeros((cluster_size, iter_size))
kmc_score_matrix_silhouette = np.zeros((cluster_size, iter_size))

# iteramos por todas as possíveis combinações dos parâmetros
i = 0
j = 0
for n_cluster in kmc_parameters["n_clusters"]:
    j = 0
    for iteration in kmc_parameters["max_iter"]:
        kmc = KMeans(n_clusters=n_cluster, max_iter=iteration).fit(X)
        kmc_labels = kmc.labels_

        # as funções que retornam as métricas de avaliação são chamadas
        cohesion    = cohesion_score(X, kmc_labels)
        separation  = separation_score(X, kmc_labels)
        entropy     = entropy_score(X, kmc_labels, label)
        silhouette  = silhouette_score(X, kmc_labels, metric='euclidean')
        
        # a pontuação de cada métrica de avaliação é armazenada em sua matriz correspondente
        kmc_score_matrix_cohesion[i, j]     = cohesion
        kmc_score_matrix_separation[i, j]   = separation
        kmc_score_matrix_entropy[i, j]      = entropy
        kmc_score_matrix_silhouette[i, j]   = silhouette

        j += 1
    i += 1

Plotamos gráficos tridimensionais para cada métrica de avaliação a fim de avaliar o comportamento de cada métrica conforme os parâmetros do classificador são modificados.

In [94]:
plotScore3D(kmc_parameters['max_iter'], kmc_parameters['n_clusters'], kmc_score_matrix_cohesion, "KMeans - Coesão\n(menor=melhor)", "Centroides", "Iterações")

plotScore3D(kmc_parameters['max_iter'], kmc_parameters['n_clusters'], kmc_score_matrix_separation, "KMeans - Separação\n(maior=melhor)", "Centroides", "Iterações")

plotScore3D(kmc_parameters['max_iter'], kmc_parameters['n_clusters'], kmc_score_matrix_entropy, "KMeans - Entropia\n(menor=melhor)", "Centroides", "Iterações")

plotScore3D(kmc_parameters['max_iter'], kmc_parameters['n_clusters'], kmc_score_matrix_silhouette, "KMeans - Silhueta\n(maior=melhor)", "Centroides", "Iterações")

<div align="center">
    <table>
        <tr>
            <td> <img src="./images/kmeans_cohesion.png" width=300/> </td>
            <td> <img src="./images/kmeans_separation.png" width=300/> </td>
        </tr>
    </table>
</div>

<div align="center">
    <table>
        <tr>
            <td> <img src="./images/kmeans_entropy.png" width=300/> </td>
            <td> <img src="./images/kmeans_silhouette.png" width=300/> </td>
        </tr>
    </table>
</div>


A fim de verificar quais são os melhores valores, estes são exibidos abaixo.

In [9]:
# Definimos o tamanho das matrizes que armazenam as pontuações
template_shape      = np.shape(kmc_score_matrix_cohesion)

# Para cada método de avaliação, definimos a melhor pontuação para aquele método.
# Para Coesão e Silhueta, as melhores pontuações são as maiores.
# Já para Separação e Entropia, são as menores.
maxScore_cohesion_kmeans   = np.min(kmc_score_matrix_cohesion)
minScore_separation_kmeans = np.max(kmc_score_matrix_separation)
minScore_entropy_kmeans    = np.min(kmc_score_matrix_entropy)
maxScore_silhouette_kmeans = np.max(kmc_score_matrix_silhouette)

# Identificada a melhor pontuação, acessamos seu índice na matriz de pontuações para definir quais parâmetros o originaram
index_cohesion   = np.unravel_index(np.argmin(kmc_score_matrix_cohesion), template_shape)
index_separation = np.unravel_index(np.argmax(kmc_score_matrix_separation), template_shape)
index_entropy    = np.unravel_index(np.argmin(kmc_score_matrix_entropy), template_shape)
index_silhouette = np.unravel_index(np.argmax(kmc_score_matrix_silhouette), template_shape)

# -------------------------------------------------------------------
#     DETERMINA MELHORES PARÂMETROS PARA CADA MÉTRICA DE AVALIAÇÃO
# -------------------------------------------------------------------
bestIter_cohesion   = kmc_parameters['max_iter'][index_cohesion[1]]
bestN_cohesion      = kmc_parameters['n_clusters'][index_cohesion[0]]

bestIter_separation = kmc_parameters['max_iter'][index_separation[1]]
bestN_separation    = kmc_parameters['n_clusters'][index_separation[0]]

bestIter_entropy    = kmc_parameters['max_iter'][index_entropy[1]]
bestN_entropy       = kmc_parameters['n_clusters'][index_entropy[0]]

bestIter_silhouette = kmc_parameters['max_iter'][index_silhouette[1]]
bestN_silhouette    = kmc_parameters['n_clusters'][index_silhouette[0]]
# -------------------------------------------------------------------
# -------------------------------------------------------------------
print("KMEANS")
print("-"*70)

# Coesão
print(f"COESÃO:\nA melhor pontuação ({maxScore_cohesion_kmeans}) foi obtida pelos parâmetros:\n {bestIter_cohesion} iterações e {bestN_cohesion} centroides.")

# Separação
print(f"SEPARAÇÃO:\nA melhor pontuação ({minScore_separation_kmeans}) foi obtida pelos parâmetros:\n {bestIter_separation} iterações e {bestN_separation} centroides.")

# Entropia
print(f"ENTROPIA:\nA melhor pontuação ({minScore_entropy_kmeans}) foi obtida pelos parâmetros:\n {bestIter_entropy} iterações e {bestN_entropy} centroides.")

# Silhueta
print(f"SILHUETA:\nA melhor pontuação ({maxScore_silhouette_kmeans}) foi obtida pelos parâmetros:\n {bestIter_silhouette} iterações e {bestN_silhouette} centroides.")

KMEANS
----------------------------------------------------------------------
COESÃO:
A melhor pontuação (1318.315995949504) foi obtida pelos parâmetros:
 1 iterações e 2 centroides.
SEPARAÇÃO:
A melhor pontuação (2556.4781878586577) foi obtida pelos parâmetros:
 1 iterações e 2 centroides.
ENTROPIA:
A melhor pontuação (0.778343872214246) foi obtida pelos parâmetros:
 1 iterações e 2 centroides.
SILHUETA:
A melhor pontuação (0.47543358614923664) foi obtida pelos parâmetros:
 1 iterações e 2 centroides.


A partir dos gráficos gerados e dos resultados exibidos, pode-se observar, no caso do KMeans, que:

* Coesão e Separação apresentam comportamentos bastante semelhantes em seus gráficos;
* A Entropia apresenta comportamento também similar à Coesão e Separação, mas tem seu maior valor com parâmetros diferentes daqueles da Coesão e Separação;
* O classificador converge rapidamente, de modo que, no eixo gráfico correspondente ao número de iterações, nota-se pouca variação, com exceção apenas da Silhueta;
* Apesar das variações, as diferentes métricas de avaliação têm parâmetros bastante similares para seus melhores valores.

---
## DBScan

Para o classificador DBScan, os parâmetros testados são:
* Tamanho do raio adotado (variável "eps")
* Número mínimo de pontos (variável "min_samples")

Verificamos que o intervalo de valores possíveis para tais parâmetros, para que se obtenha um resultado minimamente satisfatório, é bastante restrito. Muitas combinações de parâmetros faziam com que o classificador classificasse todos os pontos em uma mesma classe ou criasse um número excessivo de classes (uma combinação testada resultou em mais de 100 classes).

Devido a isso, delimitamos as combinações de parâmetros possíveis àquelas nas quais o número total de classes era igual ou menor a 8 (valor escolhido por ser o maior número de cores básicas do MatPlotLib {R,G,B,C,M,Y,B,W}, mas que é um número considerado grande, tendo em vista que originalmente tínhamos apenas 2 classes) e nos quais o ponto central do cluster superior tivesse classe distinta do ponto central do cluster inferior (assim evitando uma classificação igual para todos os pontos).

In [10]:
# a variável banana_sup_index armazena a coordenada do ponto mais próximo à coordenada (0, 1), enquanto a variável banana_inf_index o faz para a coordenada (1, -0.5). Tais coordenadas foram selecionadas a partir da visualização do gráfico do dataset, selecionando o ponto central de cada "banana"
banana_sup_index = distance.cdist(X, [[.0, 1.0]]).argmin()
banana_inf_index = distance.cdist(X, [[1.0, -0.5]]).argmin()

In [11]:
# dicionário contendo os parâmetros, onde eps varia de 0.01 a 0.15 e min_samples varia de 3 a 13
dbs_parameters = {
    "eps": [i/100 for i in range(1, 16)],
    "min_samples": [i for i in range(3, 14)]
}

# a lista validate_params armazena apenas os pares de parâmetros que atendem as condições descritas anteriormente
validated_params = []

# iteramos por todas as possíveis combinações de parâmetros e testamos as condições
for epsx in dbs_parameters["eps"]:
    j = 0
    for min_sample in dbs_parameters["min_samples"]:
        dbs = DBSCAN(eps=epsx, min_samples=min_sample).fit(X)
        dbs_labels = dbs.labels_

        # a primeira condição verifica se a label do centro da "banana" superior é diferente da label do centro da "banana" inferior
        # a segunda condição verifica se não foram geradas mais de 8 classes
        if(dbs_labels[banana_sup_index] != dbs_labels[banana_inf_index] and len(np.unique(dbs_labels)) <= 8):
            validated_params.append([epsx, min_sample])

Os pares de parâmetros que atendem as condições descritas anteriormente são exibidos. Nota-se que muitas combinações são ignoradas por não atenderem aos requisitos levantados previamente.

In [57]:
for val_params in validated_params:
    plt.plot(val_params[0], val_params[1], "k.")
    plt.xlabel("Tamanho do raio")
    plt.ylabel("Número mínimo de pontos")
plt.show()

<div align="center">
   <img src="./images/validated_params_plot.png" width=400/>
</div>

A partir da lista de parâmetros que atendem aos requisitos levantados, realiza-se o mesmo procedimento realizado anteriormente para extrair a pontuação do classificador para cada combinação de parâmetros, conforme cada métrica de avaliação.

In [12]:
# a lista é convertida em array do numpy
validated_params = np.array(validated_params)

unique_eps = np.unique(validated_params[:, 0]) 
unique_samples = np.unique(validated_params[:, 1]) 

matrix_size = ((np.shape(unique_eps)[0], np.shape(unique_samples)[0]))

dbs_score_matrix_cohesion   = np.zeros(matrix_size)
dbs_score_matrix_separation = np.zeros(matrix_size)
dbs_score_matrix_entropy    = np.zeros(matrix_size)
dbs_score_matrix_silhouette = np.zeros(matrix_size)

i = 0
j = 0
for eps in unique_eps:
    j = 0
    for sample in unique_samples:
        if [eps, sample] in validated_params.tolist():
            dbs = DBSCAN(eps=eps, min_samples=sample, n_jobs=-1).fit(X)
            dbs_labels = dbs.labels_ 

            cohesion    = cohesion_score(X, dbs_labels)
            separation  = separation_score(X, dbs_labels)
            entropy     = entropy_score(X, dbs_labels, label)
            silhouette  = silhouette_score(X, dbs_labels, metric='euclidean')
            
            dbs_score_matrix_cohesion[i, j]     = cohesion
            dbs_score_matrix_separation[i, j]   = separation
            dbs_score_matrix_entropy[i, j]      = entropy
            dbs_score_matrix_silhouette[i, j]   = silhouette
        else: 
            # como nem todos os elementos das matrizes de pontuação são válidos, os elementos inválidos são definidos como Infinito
            dbs_score_matrix_cohesion[i, j]     = float("inf")
            dbs_score_matrix_separation[i, j]   = float("inf")
            dbs_score_matrix_entropy[i, j]      = float("inf")
            dbs_score_matrix_silhouette[i, j]   = float("inf")
        j+=1
    i+=1

In [13]:
# Aqui, os elementos definidos como Infinito são alterados para 99% do valor do menor elemento da matriz. Isso é realizado a fim de facilitar o plot do gráfico tridimensional

dbs_score_matrix_cohesion_ = np.where(dbs_score_matrix_cohesion == float("inf"), np.min(dbs_score_matrix_cohesion)*.99, dbs_score_matrix_cohesion)

dbs_score_matrix_separation_ = np.where(dbs_score_matrix_separation == float("inf"), np.min(dbs_score_matrix_separation)*.99, dbs_score_matrix_separation)

dbs_score_matrix_entropy_ = np.where(dbs_score_matrix_entropy == float("inf"), np.min(dbs_score_matrix_entropy)*.99, dbs_score_matrix_entropy)

dbs_score_matrix_silhouette_ = np.where(dbs_score_matrix_silhouette == float("inf"), np.min(dbs_score_matrix_silhouette)*.99, dbs_score_matrix_silhouette)

# Já aqui, os elementos Infinito dos classificadores cuja melhor métrica é a máxima são alterados para 0, a fim de não atrapalhar a extração dos melhores parâmetros realizada abaixo
dbs_score_matrix_separation = np.where(dbs_score_matrix_cohesion == float("inf"), 0, dbs_score_matrix_cohesion)
dbs_score_matrix_silhouette = np.where(dbs_score_matrix_silhouette == float("inf"), 0, dbs_score_matrix_silhouette)

In [79]:
plotScore3D(unique_samples, unique_eps, dbs_score_matrix_cohesion_, "DBSCAN - Coesão\n(menor=melhor)", "Tamanho do raio", "Número mínimo de pontos")
plotScore3D(unique_samples, unique_eps, dbs_score_matrix_separation_, "DBSCAN - Separação\n(maior=melhor)", "Tamanho do raio", "Número mínimo de pontos")
plotScore3D(unique_samples, unique_eps, dbs_score_matrix_entropy_, "DBSCAN - Entropia", "Tamanho do raio\n(menor=melhor)", "Número mínimo de pontos")
plotScore3D(unique_samples, unique_eps, dbs_score_matrix_silhouette_, "DBSCAN - Silhueta", "Tamanho do raio\n(maior=melhor)", "Número mínimo de pontos")

<div align="center">
    <table>
        <tr>
            <td> <img src="./images/dbscan_cohesion.png" width=300/> </td>
            <td> <img src="./images/dbscan_separation.png" width=300/> </td>
        </tr>
    </table>
</div>

<div align="center">
    <table>
        <tr>
            <td> <img src="./images/dbscan_entropy.png" width=300/> </td>
            <td> <img src="./images/dbscan_silhouette.png" width=300/> </td>
        </tr>
    </table>
</div>


In [14]:
# O funcionamento dessa célula é o mesmo de sua célula equivalente do KMeans

# Definimos o tamanho das matrizes que armazenam as pontuações
template_shape = np.shape(dbs_score_matrix_cohesion)

# Para cada método de avaliação, definimos a melhor pontuação para aquele método.
# Para Coesão e Silhueta, as melhores pontuações são as maiores.
# Já para Separação e Entropia, são as menores.
maxScore_cohesion_dbs   = np.min(dbs_score_matrix_cohesion)
minScore_separation_dbs = np.max(dbs_score_matrix_separation)
minScore_entropy_dbs    = np.min(dbs_score_matrix_entropy)
maxScore_silhouette_dbs = np.max(dbs_score_matrix_silhouette)

# Identificada a melhor pontuação, acessamos seu índice na matriz de pontuações para definir quais parâmetros o originaram
index_cohesion      = np.unravel_index(np.argmin(dbs_score_matrix_cohesion), template_shape)
index_separation    = np.unravel_index(np.argmax(dbs_score_matrix_separation), template_shape)
index_entropy       = np.unravel_index(np.argmin(dbs_score_matrix_entropy), template_shape)
index_silhouette    = np.unravel_index(np.argmax(dbs_score_matrix_silhouette), template_shape)

# -------------------------------------------------------------------
#    DETERMINA MELHORES PARÂMETROS PARA CADA MÉTRICA DE AVALIAÇÃO
# -------------------------------------------------------------------
bestEps_cohesion            = unique_eps[index_cohesion[0]]
bestMinSamples_cohesion     = unique_samples[index_cohesion[1]]

bestEps_separation          = unique_eps[index_separation[0]]
bestMinSamples_separation   = unique_samples[index_separation[1]]

bestEps_entropy             = unique_eps[index_entropy[0]]
bestMinSamples_entropy      = unique_samples[index_entropy[1]]

bestEps_silhouette          = unique_eps[index_silhouette[0]]
bestMinSamples_silhouette   = unique_samples[index_silhouette[1]]
# -------------------------------------------------------------------
# -------------------------------------------------------------------

print("DBSCAN")
print("-"*70)

# Coesão
print(f"COESÃO:\nA melhor pontuação ({maxScore_cohesion_dbs}) foi obtida pelos parâmetros:\n {bestEps_cohesion} tamanho de raio e {bestMinSamples_cohesion} número mínimo de pontos.\n")

# Separação
print(f"SEPARAÇÃO:\nA melhor pontuação ({minScore_separation_dbs}) foi obtida pelos parâmetros:\n {bestEps_separation} tamanho de raio e {bestMinSamples_separation} número mínimo de pontos.\n")

# Entropia
print(f"ENTROPIA:\nA melhor pontuação ({minScore_entropy_dbs}) foi obtida pelos parâmetros:\n {bestEps_entropy} tamanho de raio e {bestMinSamples_entropy} número mínimo de pontos.\n")

# Silhueta
print(f"SILUETA:\nA melhor pontuação ({maxScore_silhouette_dbs}) foi obtida pelos parâmetros:\n {bestEps_silhouette} tamanho de raio e {bestMinSamples_silhouette} número mínimo de pontos.\n")
# -------------------------------------------------------------------

DBSCAN
----------------------------------------------------------------------
COESÃO:
A melhor pontuação (1522.3808695570162) foi obtida pelos parâmetros:
 0.08 tamanho de raio e 3.0 número mínimo de pontos.

SEPARAÇÃO:
A melhor pontuação (1957.5406314030934) foi obtida pelos parâmetros:
 0.09 tamanho de raio e 13.0 número mínimo de pontos.

ENTROPIA:
A melhor pontuação (0.12317248547937495) foi obtida pelos parâmetros:
 0.11 tamanho de raio e 10.0 número mínimo de pontos.

SILUETA:
A melhor pontuação (0.2516764424216868) foi obtida pelos parâmetros:
 0.11 tamanho de raio e 10.0 número mínimo de pontos.



A partir dos gráficos gerados e dos resultados exibidos, pode-se observar, no caso do DBSCAN, que:

* Coesão e Separação apresentam gráficos ligeiramente semelhantes, com picos e vales em posições similares;
* A Silhueta mostra, pelo seu gráfico, ser uma ocmbinação da Separação e Silhueta, visto que seus picos e vales correspondem ora a picos e vales da Coesão, ora da Silhueta;
* A Entropia apresenta um pico que representa uma região de entropia extremamente elevada, mas mantém valores baixos para as demais combinações de parâmetros;
* Não houve um consenso entre as diferentes métricas de quais seriam os melhores parâmetros. Ainda que Entropia e Silhueta tenham chegado aos mesmo parâmetros, as demais métricas chegaram a valores significativamente distintos.


---
## AGNES

In [15]:
linkage_dict = {
    0: "ward",
    1: "complete",
    2: "single"
}

agc_parameters = {
    'n_clusters': [i for i in range(2, 9)],
    'linkage': [0, 1, 2]
}

cluster_size = np.shape(agc_parameters['n_clusters'])[0]
linkage_size = np.shape(agc_parameters['linkage'])[0]

agc_score_matrix_cohesion   = np.zeros((cluster_size, linkage_size))
agc_score_matrix_separation = np.zeros((cluster_size, linkage_size))
agc_score_matrix_entropy    = np.zeros((cluster_size, linkage_size))
agc_score_matrix_silhouette = np.zeros((cluster_size, linkage_size))

i = 0
j = 0
for n_cluster in agc_parameters['n_clusters']:
    j = 0
    for linkage in agc_parameters['linkage']:
        agc = AgglomerativeClustering(linkage=linkage_dict[linkage], n_clusters=n_cluster).fit(X)

        # agc_labels = np.where(agc.labels_ == 1, 0, 1)
        agc_labels = agc.labels_ 

        cohesion    = cohesion_score(X, agc_labels)
        separation  = separation_score(X, agc_labels)
        entropy     = entropy_score(X, agc_labels, label)
        silhouette  = silhouette_score(X, agc_labels, metric='euclidean')

        agc_score_matrix_cohesion[i, j]     = cohesion
        agc_score_matrix_separation[i, j]   = separation
        agc_score_matrix_entropy[i, j]      = entropy
        agc_score_matrix_silhouette[i, j]   = silhouette

        j += 1
    i += 1

In [95]:
plotScore3D(agc_parameters['linkage'], agc_parameters['n_clusters'], agc_score_matrix_cohesion, "AGNES - Coesão\n(menor=melhor)", "Número de clusters", "Similaridade\n0-ward, 1-complete, 2-single")

plotScore3D(agc_parameters['linkage'], agc_parameters['n_clusters'], agc_score_matrix_separation, "AGNES - Separação\n(maior=melhor)", "Número de clusters", "Similaridade\n0-ward, 1-complete, 2-single")

plotScore3D(agc_parameters['linkage'], agc_parameters['n_clusters'], agc_score_matrix_entropy, "AGNES - Entropia\n(menor=melhor)", "Número de clusters", "Similaridade\n0-ward, 1-complete, 2-single")

plotScore3D(agc_parameters['linkage'], agc_parameters['n_clusters'], agc_score_matrix_silhouette, "AGNES - Silhueta\n(maior=melhor)", "Número de clusters", "Similaridade\n0-ward, 1-complete, 2-single")

<div align="center">
    <table>
        <tr>
            <td> <img src="./images/agnes_cohesion.png" width=300/> </td>
            <td> <img src="./images/agnes_separation.png" width=300/> </td>
        </tr>
    </table>
</div>

<div align="center">
    <table>
        <tr>
            <td> <img src="./images/agnes_entropy.png" width=300/> </td>
            <td> <img src="./images/agnes_silhouette.png" width=300/> </td>
        </tr>
    </table>
</div>


In [16]:
# O funcionamento dessa célula é o mesmo de sua célula equivalente do KMeans

# Definimos o tamanho das matrizes que armazenam as pontuações
template_shape      = np.shape(agc_score_matrix_cohesion)

# Para cada método de avaliação, definimos a melhor pontuação para aquele método.
# Para Coesão e Silhueta, as melhores pontuações são as maiores.
# Já para Separação e Entropia, são as menores.
maxScore_cohesion_ag   = np.min(agc_score_matrix_cohesion)
minScore_separation_ag = np.max(agc_score_matrix_separation)
minScore_entropy_ag    = np.min(agc_score_matrix_entropy)
maxScore_silhouette_ag = np.max(agc_score_matrix_silhouette)

# Identificada a melhor pontuação, acessamos seu índice na matriz de pontuações para definir quais parâmetros o originaram
index_cohesion      = np.unravel_index(np.argmin(agc_score_matrix_cohesion), template_shape)
index_separation    = np.unravel_index(np.argmax(agc_score_matrix_separation), template_shape)
index_entropy       = np.unravel_index(np.argmin(agc_score_matrix_entropy), template_shape)
index_silhouette    = np.unravel_index(np.argmax(agc_score_matrix_silhouette), template_shape)

# -------------------------------------------------------------------
#    DETERMINA MELHORES PARÂMETROS PARA CADA MÉTRICA DE AVALIAÇÃO
# -------------------------------------------------------------------
bestNAg_cohesion            = agc_parameters['n_clusters'][index_cohesion[0]]
bestLinkage_cohesion     = agc_parameters['linkage'][index_cohesion[1]]

bestNAg_separation          = agc_parameters['n_clusters'][index_separation[0]]
bestLinkage_separation   = agc_parameters['linkage'][index_separation[1]]

bestNAg_entropy             = agc_parameters['n_clusters'][index_entropy[0]]
bestLinkage_entropy      = agc_parameters['linkage'][index_entropy[1]]

bestNAg_silhouette          = agc_parameters['n_clusters'][index_silhouette[0]]
bestLinkage_silhouette   = agc_parameters['linkage'][index_silhouette[1]]
# -------------------------------------------------------------------
# -------------------------------------------------------------------
print("AGNES")
print("-"*70)

# Coesão
print(f"COESÃO:\nA melhor pontuação ({maxScore_cohesion_ag}) foi obtida pelos parâmetros:\n {bestNAg_cohesion} número de clusters e {linkage_dict[bestLinkage_cohesion]} medida de similaridade.\n")

# Separação
print(f"SEPARAÇÃO:\nA melhor pontuação ({minScore_separation_ag}) foi obtida pelos parâmetros:\n {bestNAg_separation} número de clusters e {linkage_dict[bestLinkage_separation]} medida de similaridade.\n")

# Entropia
print(f"ENTROPIA:\nA melhor pontuação ({minScore_entropy_ag}) foi obtida pelos parâmetros:\n {bestNAg_entropy} número de clusters e {linkage_dict[bestLinkage_entropy]} medida de similaridade.\n")

# Silhueta
print(f"SILUETA:\nA melhor pontuação ({maxScore_silhouette_ag}) foi obtida pelos parâmetros:\n {bestNAg_silhouette} número de clusters e {linkage_dict[bestLinkage_silhouette]} medida de similaridade.\n")

AGNES
----------------------------------------------------------------------
COESÃO:
A melhor pontuação (2874.460727893919) foi obtida pelos parâmetros:
 2 número de clusters e single medida de similaridade.

SEPARAÇÃO:
A melhor pontuação (104.94433747931967) foi obtida pelos parâmetros:
 2 número de clusters e single medida de similaridade.

ENTROPIA:
A melhor pontuação (0.005703878868730569) foi obtida pelos parâmetros:
 2 número de clusters e single medida de similaridade.

SILUETA:
A melhor pontuação (0.4651893317755256) foi obtida pelos parâmetros:
 2 número de clusters e ward medida de similaridade.



----

## Comparação dos classificadores

In [17]:
cohesion_row = [maxScore_cohesion_kmeans,maxScore_cohesion_dbs,maxScore_cohesion_ag]

separation_row = [minScore_separation_kmeans,minScore_separation_dbs,minScore_separation_ag]

entropy_row = [minScore_entropy_kmeans,minScore_entropy_dbs,minScore_entropy_ag]

silhouette_row = [maxScore_silhouette_kmeans,maxScore_silhouette_dbs,maxScore_silhouette_ag]

storage = [
    cohesion_row,
    separation_row,
    entropy_row,
    silhouette_row
]

pd.DataFrame(storage, index=["Coesão", "Separação", "Entropia", "Silhueta"], columns=["KMeans", "DBScan", "Agnes"])

,KMeans,DBScan,Agnes
Coesão,1318.316,1522.381,2874.461
Separação,2556.478,1957.541,104.944
Entropia,0.778,0.123,0.006
Silhueta,0.475,0.252,0.465


De acordo com o critério de Entropia, o melhor classificador é o Agnes, cuja classificação é exibida abaixo.

In [105]:
agc = AgglomerativeClustering(linkage="single", n_clusters=2).fit(X)
agc_labels = agc.labels_

color = ["r.", "g.", "b.", "y.", "m.", "c.", "w.", "k."]
for i in range(len(X)):
    plt.plot(X[i][0], X[i][1], color[agc_labels[i]])
plt.title("AGNES")
plt.show()

<div align="center">
    <img src="./images/best_agnes.png" width=300/>
</div>

Já de acordo com os critérios de Coesão, Separação e Silhueta, o melhor classificador é o KMeans, cuja classificação é exibida abaixo.

In [104]:
kmc = KMeans(n_clusters=2, max_iter=1).fit(X)
kmc_labels = kmc.labels_

color = ["r.", "g.", "b.", "y.", "m.", "c.", "w.", "k."]
for i in range(len(X)):
    plt.plot(X[i][0], X[i][1], color[kmc_labels[i]])
plt.title("KMeans")
plt.show()

<div align="center">
    <img src="./images/best_kmeans.png" width=300/>
</div>

Todavia, ao observarmos as melhores classificações do classificador DBScan, abaixo, podemos notar resultados significativamente distintos:

In [111]:
dbs = DBSCAN(eps=.11, min_samples=10, n_jobs=-1).fit(X)
dbs_labels = dbs.labels_

color = ["r.", "g.", "b.", "y.", "m.", "c.", "w.", "k."]
for i in range(len(X)):
    plt.plot(X[i][0], X[i][1], color[dbs_labels[i]])
plt.title("DBScan")
plt.show()

<div align="center">
    <img src="./images/best_dbscan.png" width=300/>
</div>

---
## Hipóteses Geradas

### Premissa

Durante todo o processo, como implementamos as funções (Coesão, Separação e Entropia) na mão, as verificamos múltiplas vezes e temos certeza que estão funcionando conforme as explicações dadas em aula. Nesse sentido, concluímos que os resultados inusitados não sejam ocasionados por um problema de implementação, mas sim por outros fatores.

---

### Hipótese

Antes de falar dos valores, a nossa hipótese é que as métricas de avaliação utilizadas não são as mais adequadas ao nosso dataset sem que sejam realizadas modificações no resultado de seus agrupamentos. Acreditamos que isso ocorra por se tratar de um dataset cujos clusters originais são alongados e bastante próximos (o que dificulta a obtenção de bons agrupamentos ao se utilizar Separação, Coesão ou Silhueta como métrica) e com muitos pontos de ruído entre eles (o que dificulta no caso da Entropia).

Segue a imagem de uma tabela que elenca os melhores resultados de cada métrica de avaliação para cada metódo de agrupamento, informando os parâmetros utilizados para se chegar a tal métrica.

<div align="center">
    <img src="./images/fig_01.png" width=300/>
</div>

O "problema" aparece quando plotamos cada um dos possíveis resultados presentes nessa tabela. Sendo, o "problema" em questão, a grande diferença entre o resultado dos metódos de agrupamento que supostamente deveriam ser os melhores quando comparados com o agrupamento do dataset original. Matematicamente, sabemos que o "grau de semelhança" entre cada um dos pontos agrupados com o seu agrupamento original é, na realidade, a acurácia. Nesse sentido, para cada um dos plots dos melhores metódos de agrupamento da tabela acima, mostramos no gráfico qual sua acurácia (ainda que tal métrica não tenha sido pedida no trabalho e não seja adequada para situações de aprendizado não-supervisionado).

O melhor KMeans para Coesão e Separação é o KMeans abaixo. Como ele tem 8 clusters distintos, fizemos um teste agrupando os clusters inferiores em um só, e os superiores em outro, cujo resultado é a imagem ao lado. Ao realizar esse agrupamento ad hoc (visto que a ordenação das classes ocorre de modo aleatório), conseguimos um resultado excelente para o metódo de agrupamento. Todavia, sem tal agrupamento, o resultado obtido é bastante pobre.

<div align="center">
    <table>
        <tr>
            <td> <img src="./images/fig_02.png" width=300/> </td>
            <td> <img src="./images/fig_03.png" width=300/> </td>
        </tr>
    </table>
</div>

Comparando o melhor KMeans para estas duas métricas com os demais metódos de agrupamento para as mesmas métricas, vemos que o melhor DBScan da Coesão (abaixo) também realiza o agrupamento em vários clusters distintos, e que, mesmo agrupando-os como feito acima, o resultado é inferior ao do KMeans.

<div align="center">
    <table>
        <tr>
            <td> <img src="./images/fig_04.png" width=300/> </td>
            <td> <img src="./images/fig_05.png" width=300/> </td>
        </tr>
    </table>
</div>

Já para o DBScan da Separação, o resultado é bastante satisfatório do ponto de vista da acurácia. Entretanto, sua Separabilidade é bastante inferior à do KMeans acima (como pode ser visto na tabela).

<div align="center">
    <img src="./images/fig_06.png" width=300/>
</div>

Para o Agnes com melhores Coesão e Separabilidade, também se chega a um número grande de classes, conforme figura abaixo. Ao realizar o agrupamento, entretanto, chega-se à Acurácia mais alta obtida até então.

<div align="center">
    <table>
        <tr>
            <td> <img src="./images/fig_07.png" width=300/> </td>
            <td> <img src="./images/fig_08.png" width=300/> </td>
        </tr>
    </table>
</div>

A partir disso, verificamos que os critérios de Coesão e Separabilidade, por si sós, não são boas métricas para nosso dataset, visto que o melhor metódo de agrupamento para elas (o primeiro KMeans) apresentou um resultado bastante pobre, o qual mesmo após agrupamento não é o melhor resultado do ponto de vista da similaridade com o dataset original. Além disso, notamos que os resultados que obtiveram um grande número de classes só apresentaram bom resultado após o agrupamento, o qual não foi necessário para o DBScan com melhor Coesão. Consideramos isso bastante estranho, visto que, de todos estes metódos de agrupamento, o DBScan da Coesão é o que tem a pior Coesão, mas é o qual, sem agrupamento posterior, realiza a melhor classificação inicial.

Já quanto à Entropia, o melhor resultado foi do AGNES cujo resultado é mostrado abaixo, ao lado do melhor DBScan para Entropia e melhor KMeans para Entropia.

<div align="center">
    <table>
        <tr>
            <td> <img src="./images/fig_09.png" width=300/> </td>
            <td> <img src="./images/fig_10.png" width=300/> </td>
        </tr>
    </table>
</div>

<div align="center">
    <img src="./images/fig_11.png" width=300/>
</div>

Percebe-se que o DBScan com maior Entropia é um metódo de agrupamento muito melhor que o AGNES de maior Entropia.

Já para a métrica de Silhueta, o melhor resultado é o do KMeans exibido acima. Novamente, o DBScan de maior Silhueta apresenta resultado muito melhor, enquanto o AGNES de melhor Silhueta (abaixo) apresenta resultado bastante similar ao do KMeans, apresentando acurácia maior, ainda que seu valor de Silhueta tenha sido menor que a do KMeans.

<div align="center">
    <img src="./images/fig_12.png" width=300/>
</div>

A partir do exposto, não tínhamos certeza de como proceder, tendo em vista que os melhores metódos de agrupamento para cada métrica (exibidos abaixo novamente para destaque) não são os melhores metódos de agrupamento obtidos.

<div align="center">
    <table>
        <tr>
            <td> <img src="./images/fig_13.png" width=300/> </td>
            <td> <img src="./images/fig_14.png" width=300/> </td>
        </tr>
    </table>
</div>

<div align="center">
    <img src="./images/fig_15.png" width=300/>
</div>

---

### Conclusão

Acreditamos que é válida a análise de que tais métricas não são, na realidade, as melhores métricas para este dataset.

Além disso, tudo indica que realizar o agrupamento de modo ad hoc para casos em que há mais de 2 clusters ou outra base não é válido, visto que essas abordagens são caracterizadas como aprendizagem não supervisionado, de modo que utilizamos o agrupamento apenas para experimentação.